In [232]:
# 라이브러리 로드
import pandas as pd
import numpy as np

In [233]:
# 데이터 로드

수익성 = pd.read_csv('./datasets/코스닥/코스닥 수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./datasets/코스닥/코스닥 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./datasets/코스닥/코스닥 활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./datasets/코스닥/코스닥 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./datasets/코스닥/코스닥 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./datasets/코스닥/코스닥 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./datasets/코스닥/에비~ 코스닥.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./datasets/코스닥/per pbr 총부채 코스닥.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./datasets/코스닥/코스닥 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./datasets/코스닥/코스닥시가총액.csv',index_col=0)  #### 시총도 붙여야됨
단기차입금 = pd.read_csv('./datasets/코스닥/코스닥 단기차입금 유동성 장기부채.csv',encoding='cp949')
거시변수 = pd.read_excel('./datasets/거시경제 변수/거시변수.xlsx', engine="openpyxl")
상장일 = pd.read_csv('./datasets/코스닥/코스닥 상장일.csv',encoding='cp949')
대분류 = pd.read_csv('./datasets/코스닥/코스닥 대분류.csv',encoding='cp949')
유형자산 = pd.read_csv("./datasets/코스닥/코스닥 유형자산증가율.csv", encoding= "cp949")
현금성자산 = pd.read_csv("./datasets/코스닥/코스닥 현금성 비율.csv", encoding= "cp949")

In [234]:
#  사용 안하는 코드 컬럼 삭제 
대분류.drop(['통계청 한국표준산업분류 코드 10차(대분류)'],axis=1,inplace=True)

In [235]:
#  데이터 결합

df = 안정성.merge(수익성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(활동성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(성장성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금흐름,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(연구개발,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(EBIT,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(per_pbr_총부채,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(직원수_업력,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(단기차입금,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(상장일,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(대분류,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(유형자산,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금성자산,on=['회사명','거래소코드','회계년도'],how='left')

In [236]:
len(df.columns)

99

In [237]:
## 거래소코드 패딩
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
## 외국계기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

# 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

def func_not(x):
    return not x

def change_year_2(x):
    return (x[:4])

In [238]:
## 거래소 코드 패딩
df['거래소코드'] = df['거래소코드'].map(padding)
# 회계년도 제거
df['회계년도'] = df['회계년도'].map(change_year)

In [239]:
df.isna().sum()

회사명                             0
거래소코드                           0
회계년도                            0
자산(*)(요약)(IFRS)              1844
자본(*)(요약)(IFRS)              1844
                             ... 
통계청 한국표준산업분류 10차(대분류)           0
유형자산의 증가(*)(IFRS)(천원)        1844
유형자산의 증가(*)(IFRS연결)(천원)      5530
현금및현금성자산(요약)(IFRS)(백만원)      1844
현금및현금성자산(요약)(IFRS연결)(백만원)    5530
Length: 99, dtype: int64

In [240]:
## 중복되는 데이터 제거
df.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

## 없음


In [241]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 15617 개
전체 기업 수 1744


In [242]:
## 외국계 기업 제거
print("제거할 데이터 수 : ", len(df[df["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
print('제거할 외국계 기업 수',len(df[~df["거래소코드"].map(func_isnt_fcompany)].회사명.unique()))

df = df[df["거래소코드"].map(func_isnt_fcompany)]
print('남은 데이터 수',df.shape[0]) # 외국계기업 데이터 24개 제거
print('남은 기업 수',len(df['회사명'].unique()))

제거할 데이터 수 :  152
제거할 외국계 기업 수 23
남은 데이터 수 15465
남은 기업 수 1721


In [243]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df["회사명"].unique()))
print("거래소코드수 : ", len(df["거래소코드"].unique()))

회사수 :  1721
거래소코드수 :  1743


In [244]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df["회사명"].unique()):
    if len(df[df["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")

거래소코드가 2개인 기업 수 22 개


In [245]:
list_overlap_code

['(주)본느',
 '(주)세화피앤씨',
 '(주)아이엘사이언스',
 '(주)알로이스',
 '(주)에스에이티',
 '(주)엔케이맥스',
 '(주)원바이오젠',
 '(주)인산가',
 '(주)자비스',
 '(주)정다운',
 '(주)카이노스메드',
 '(주)티에스트릴리온',
 '(주)포인트엔지니어링',
 '(주)한국비엔씨',
 '(주)휴럼',
 '나무기술(주)',
 '소프트캠프(주)',
 '씨아이에스(주)',
 '에스케이오션플랜트(주)',
 '엠에프엠코리아(주)',
 '줌인터넷(주)',
 '패션플랫폼(주)']

In [246]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df[df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df[df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [247]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df["거래소코드"].unique()):
    if len(df[df["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [248]:
## 연결 결측치 개별로 채우기
for i in range(df.shape[0]):
    if pd.isnull(df.loc[i,'유보율(IFRS연결)(백만원)']):
       df.loc[i,'유보율(IFRS연결)(백만원)'] = df.loc[i,'유보율(IFRS)']
    if pd.isnull(df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'자산(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'부채(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]자본(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'당기순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융원가(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융원가(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융원가(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융수익(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융수익(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융수익(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)'] = df.loc[i,'매출액(수익)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'연구개발비(IFRS연결)(천원)']):
       df.loc[i,'연구개발비(IFRS연결)(천원)'] = df.loc[i,'연구개발비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      유형자산(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자본금회전률(IFRS연결)']):
       df.loc[i,'자본금회전률(IFRS연결)'] = df.loc[i,'자본금회전률(IFRS)']
    if pd.isnull(df.loc[i,'총자본회전률(IFRS연결)']):
       df.loc[i,'총자본회전률(IFRS연결)'] = df.loc[i,'총자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'재고자산회전률(IFRS연결)']):
       df.loc[i,'재고자산회전률(IFRS연결)'] = df.loc[i,'재고자산회전률(IFRS)']
    if pd.isnull(df.loc[i,'유형자산회전율(IFRS연결)']):
       df.loc[i,'유형자산회전율(IFRS연결)'] = df.loc[i,'유형자산회전율(IFRS)']
    if pd.isnull(df.loc[i,'자기자본회전률(IFRS연결)']):
       df.loc[i,'자기자본회전률(IFRS연결)'] = df.loc[i,'자기자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'타인자본회전률(IFRS연결)']):
       df.loc[i,'타인자본회전률(IFRS연결)'] = df.loc[i,'타인자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'경영자본순이익률(IFRS연결)']):
       df.loc[i,'경영자본순이익률(IFRS연결)'] = df.loc[i,'경영자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'금융비용부담률(IFRS연결)']):
       df.loc[i,'금융비용부담률(IFRS연결)'] = df.loc[i,'금융비용부담률(IFRS)']
    if pd.isnull(df.loc[i,'총자본순이익률(IFRS연결)']):
       df.loc[i,'총자본순이익률(IFRS연결)'] = df.loc[i,'총자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'매출액총이익률(IFRS연결)']):
       df.loc[i,'매출액총이익률(IFRS연결)'] = df.loc[i,'매출액총이익률(IFRS)']
    if pd.isnull(df.loc[i,'자기자본순이익률(IFRS연결)']):
       df.loc[i,'자기자본순이익률(IFRS연결)'] = df.loc[i,'자기자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'총자본정상영업이익률(IFRS연결)']):
       df.loc[i,'총자본정상영업이익률(IFRS연결)'] = df.loc[i,'총자본정상영업이익률(IFRS)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)'] = df.loc[i,'자산(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'자본(*)(요약)(IFRS연결)']):
       df.loc[i,'자본(*)(요약)(IFRS연결)'] = df.loc[i,'자본(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'현금비율(IFRS연결)(백만원)']):
       df.loc[i,'현금비율(IFRS연결)(백만원)'] = df.loc[i,'현금비율(IFRS)']
    if pd.isnull(df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'차입금의존도(IFRS연결)']):
       df.loc[i,'차입금의존도(IFRS연결)'] = df.loc[i,'차입금의존도(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'감가상각비(IFRS연결)(천원)']):
       df.loc[i,'감가상각비(IFRS연결)(천원)'] = df.loc[i,'감가상각비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'부채비율(IFRS연결)']):
       df.loc[i,'부채비율(IFRS연결)'] = df.loc[i,'부채비율(IFRS)']
    if pd.isnull(df.loc[i,'당좌비율(IFRS연결)']):
       df.loc[i,'당좌비율(IFRS연결)'] = df.loc[i,'당좌비율(IFRS)']
    if pd.isnull(df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)']):
       df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)'] = df.loc[i,'*이자비용(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'순운전자본비율(IFRS연결)']):
       df.loc[i,'순운전자본비율(IFRS연결)'] = df.loc[i,'순운전자본비율(IFRS)']
    if pd.isnull(df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)']):
       df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)'] = df.loc[i,'유동성장기부채(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)']):
       df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)'] = df.loc[i,'단기차입금(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'유형자산의 증가(*)(IFRS연결)(천원)']):
       df.loc[i,'유형자산의 증가(*)(IFRS연결)(천원)'] = df.loc[i,'유형자산의 증가(*)(IFRS)(천원)']
    if pd.isnull(df.loc[i,'현금및현금성자산(요약)(IFRS연결)(백만원)']):
        df.loc[i,'현금및현금성자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'현금및현금성자산(요약)(IFRS)(백만원)']


In [249]:
# #  연결 재무제표를 개별제무제표로 대체
# 위의 코드 요약해서 다시 세워본 코드 


# connected_cols = [col for col in df.columns if "연결" in col]

# for connected_col in connected_cols:
#     original_col = connected_col.replace("연결", "")
#     if original_col in df.columns:
#         df.loc[df[connected_col].isnull(), connected_col] = df[original_col]

In [250]:
#  삭제 할 개별재무제표 컬럼 정리  

drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'경영자본순이익률(IFRS)','금융비용부담률(IFRS)','총자본순이익률(IFRS)','매출액총이익률(IFRS)',
'자기자본순이익률(IFRS)','총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x',
'유동성장기부채(요약)(IFRS)(백만원)','단기차입금(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS연결)(백만원)','[제조]자본(*)(요약)(IFRS연결)(백만원)',"유형자산의 증가(*)(IFRS)(천원)",'현금및현금성자산(요약)(IFRS)(백만원)'
]

# 개별재무제표 컬럼 삭제
df.drop(drop_col,axis=1,inplace=True)

In [251]:
len(df.columns)

49

In [252]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '자본(*)(요약)(IFRS연결)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -20, 2012년 -10) -> -10-(-20) / np.abs(-20)= 
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1))*100
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '자본(*)(요약)(IFRS연결)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(['[제조]      유형자산(요약)(IFRS연결)(백만원)','[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'],axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [253]:
## 시가총액 merge
시가총액['회계년도'] = 시가총액['회계년도'].astype(str).str[:4].astype(int)
df = df.merge(시가총액,on=['거래소코드','회계년도'],how='left')
df['시가총액(백만원)'] = df['시가총액'] / 1000000
df.drop(['시가총액'],axis=1,inplace=True)

## 거시변수 merge
거시변수.rename(columns={'회계연도':'회계년도'},inplace=True)
df = df.merge(거시변수,on=['회계년도'],how='left')

In [254]:
### 파생변수 만들기 


## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)

## 누적수익성비율  - 자산0이면 inf - 공시안함 
df['누적수익성비율'] = df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']

## 이자부담율
# df['이자부담률'] = df['EBITDA(IFRS)(백만원)'] / df['*이자비용(요약)(IFRS연결)(백만원)'] 

## 부채상환계수 EBITDA / (이자비용 + 단기차입금 + 유동성장기부채)
# df['부채상환계수'] = df['EBITDA(IFRS)(백만원)'] / (df['*이자비용(요약)(IFRS연결)(백만원)']+df['단기차입금(요약)(IFRS연결)(백만원)'] + df['유동성장기부채(요약)(IFRS연결)(백만원)'])

## 현금흐름_자산 = 영업현금흐름 / 당기총자산
# df['현금흐름 대 자산'] = df.groupby(['회사명','거래소코드'])['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].shift(0) / df.groupby(['회사명','거래소코드'])['자산(*)(요약)(IFRS연결)'].shift(0)

## 현금흐름_매출액
# df['현금흐름 대 매출액'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

# 토빈Q
# df['토빈Q'] = (df['부채(*)(요약)(IFRS연결)(백만원)'] + df['시가총액(백만원)']) / df['자산(*)(요약)(IFRS연결)']

## TMD
df['TMD'] = df['부채(*)(요약)(IFRS연결)(백만원)'] / df['시가총액(백만원)']

## 알트만 변수
df['시총/총자산'] = df['시가총액(백만원)'] / df['자산(*)(요약)(IFRS연결)']

## RD -단위도 수정
# df['연구개발비(IFRS연결)(백만원)'] = df['연구개발비(IFRS연결)(천원)']/1000
# df.drop(['연구개발비(IFRS연결)(천원)'],axis=1,inplace=True)
# df['RD'] = df['연구개발비(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

## 직원수변동률 , 직원인건비총액(백만원단위) (고민필요)
# df['직원수변동률%'] = df.groupby(['회사명','거래소코드'])['합계_기말인원(명)'].pct_change()*100

## 직원인건비총액(백만원)
# df['직원인건비총액(백만원)'] = (df['합계_년간총급여(천원)']/1000) * df['합계_기말인원(명)']
# df.drop(['합계_년간총급여(천원)'],axis=1,inplace=True)

## 알트만 변수  EBIT = 법인세비용차감전순이익+금융원가-금융수익
df['EBIT/총자산'] = (df['법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] + df['금융원가(요약)(IFRS연결)(백만원)']-df['금융수익(요약)(IFRS연결)(백만원)'])/df['자산(*)(요약)(IFRS연결)']

## PBR 
# df['PBR'] = df['시가총액(백만원)']  / df['자본(*)(요약)(IFRS연결)']


# 영업손익 전기영업손익 + 당기영업손익
df["당기전기영업손익"] = (df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(0) + df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(1)) / 2

# capex컬럼 단위 맞춰주기
# df["유형자산의 증가(*)(IFRS연결)(천원)"] = df["유형자산의 증가(*)(IFRS연결)(천원)"] / 1000

# findep =  (capex - 영업현금흐름 ) / capex
# df["FINDEP"] = df["유형자산의 증가(*)(IFRS연결)(천원)"] - df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']

# PBR 변화율
# df['PBR변화율']  = (df.groupby(['회사명', '거래소코드'])['PBR'].shift(0) - df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) *100

# ROA 변화율
# df['ROA변화율']  = (df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(0) - df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(1)) *100

# EBITDA마진율
# df["EBITDA마진율"] = (df["EBITDA(IFRS)(백만원)"] / df["매출액(수익)(요약)(IFRS연결)(백만원)"]) * 100

# 논문을 바탕으로 새로운 컬럼 생성

# df['현금흐름/총부채비율'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['부채(*)(요약)(IFRS연결)(백만원)']
# df["영업현금흐름-단기차입금"] = df["[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)"] - df["단기차입금(요약)(IFRS연결)(백만원)"]
df["abs(영업현금흐름-당기순이익)/매출액"] = np.abs((df["[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)"] - df["당기순이익(손실)(요약)(IFRS연결)(백만원)"])) / df["매출액(수익)(요약)(IFRS연결)(백만원)"]



In [255]:
len(df.columns)

61

In [256]:
# 컬럼이름 정리

rename_list = {'자산(*)(요약)(IFRS연결)':'자산', '자본(*)(요약)(IFRS연결)':'자본',
       '차입금의존도(IFRS연결)':'차입금의존도', '유보율(IFRS연결)(백만원)':'유보율', '현금비율(IFRS연결)(백만원)':'현금비율',
       '부채비율(IFRS연결)':'부채비율', '당좌비율(IFRS연결)':'당좌비율', '순운전자본비율(IFRS연결)':'순운전자본비율',
       '*이자비용(요약)(IFRS연결)(백만원)':'이자비용', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'영업손익',
       '감가상각비(IFRS연결)(천원)':'감가상각비', 'EBITDA(IFRS)(백만원)':'EBITDA', '총자본정상영업이익률(IFRS연결)':'총자본정상영업이익률',
       '자기자본순이익률(IFRS연결)':'자기자본순이익률', '총자본순이익률(IFRS연결)':'총자본순이익률', '매출액총이익률(IFRS연결)':'매출액총이익률',
       '금융비용부담률(IFRS연결)':'금융비용부담률', '경영자본순이익률(IFRS연결)':'경영자본순이익률', '자본금회전률(IFRS연결)':'자본금회전률',
       '유형자산회전율(IFRS연결)':'유형자산회전율', '재고자산회전률(IFRS연결)':'재고자산회전률', '자기자본회전률(IFRS연결)':'자기자본회전률',
       '타인자본회전률(IFRS연결)':'타인자본회전률', '총자본회전률(IFRS연결)':'총자본회전률', 
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)':'이익잉여금',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)':'영업활동으로 인한 현금흐름',
       '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액', '금융원가(요약)(IFRS연결)(백만원)':'금융원가',
       '금융수익(요약)(IFRS연결)(백만원)':'금융수익', '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)':'법인세비용차감전순이익',
       '당기순이익(손실)(요약)(IFRS연결)(백만원)':'당기순이익', 
       '부채(*)(요약)(IFRS연결)(백만원)':'부채', '외국인_주식분포비율':'외국인지분율', '보통주식비율':'대주주지분율',
       '단기차입금(요약)(IFRS연결)(백만원)':'단기차입금', '유동성장기부채(요약)(IFRS연결)(백만원)':'유동성장기부채',
       '시가총액(백만원)':'시가총액','연구개발비(IFRS연결)(백만원)':'연구개발비','직원인건비총액(백만원)':'직원인건비총액','통계청 한국표준산업분류 10차(대분류)':'산업군',
       "유형자산의 증가(*)(IFRS연결)(천원)" : "CAPEX" }

df.rename(columns=rename_list,inplace=True)

In [257]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '자본', '차입금의존도', '유보율', '현금비율', '부채비율',
       '당좌비율', '순운전자본비율', '이자비용', '영업손익', '감가상각비', 'EBITDA', '총자본정상영업이익률',
       '자기자본순이익률', '총자본순이익률', '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률',
       '유형자산회전율', '재고자산회전률', '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금',
       '영업활동으로 인한 현금흐름', '매출액', '연구개발비(IFRS연결)(천원)', '금융원가', '금융수익',
       '법인세비용차감전순이익', '당기순이익', '부채', '합계_기말인원(명)', '합계_년간총급여(천원)', '외국인지분율',
       '대주주지분율', '단기차입금', '유동성장기부채', '상장일', '산업군', 'CAPEX',
       '현금및현금성자산(요약)(IFRS연결)(백만원)', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율',
       '정상영업이익증가율', '시가총액', '장단기금리차', 'cpi', '업력', '누적수익성비율', 'TMD', '시총/총자산',
       'EBIT/총자산', '당기전기영업손익', 'abs(영업현금흐름-당기순이익)/매출액'],
      dtype='object')

In [258]:
## 회계년도 2011년도 제거
df['상장일'] = pd.to_datetime(df['상장일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)


df = df[df['회계년도']>2011].reset_index(drop=True)

print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))


전체 행: 14077 개
전체 기업 수 1684


In [259]:
df.isna().sum()

회사명                         0
거래소코드                       0
회계년도                        0
자산                       1456
자본                       1456
                         ... 
TMD                      2618
시총/총자산                   2618
EBIT/총자산                 1457
당기전기영업손익                 2085
abs(영업현금흐름-당기순이익)/매출액    1459
Length: 61, dtype: int64

-----

## 타겟 라벨링
-----


In [260]:
#  데이터 로드, 필요 없는 컬럼 삭제

부실 = pd.read_csv('./datasets/y/y값최종.csv',index_col=0)
부실['회계년도'] = 부실['회계년도'].astype(int)
부실['거래소코드'] = 부실['거래소코드'].map(padding)
부실.drop(['번호','회사명'],axis=1,inplace=True)

In [261]:
# 데이터 결합
df = df.merge(부실,on=['회계년도','거래소코드'],how='left')

In [262]:
## 상폐기업 라벨링

df["공시제목"].fillna(0,inplace=True)

상폐_ind = df[df["공시제목"]== '상장폐지기업'].index

In [263]:
df.loc[상폐_ind,"target"] = 1


### set1
df.loc[상폐_ind,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) :
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

## set_3

df.loc[상폐_ind,"target_2"] = 1

for i in df.index:
    if (df.loc[i,"target_2"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-1, "target_2"] = 1
df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)


## set_3

df.loc[상폐_ind,"target_3"] = 1

for i in df.index:
    if (df.loc[i,"target_3"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-1, "target_3"] = 1
df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)

In [264]:
print(df.target_1.sum())
print(df.target_2.sum())
print(df.target_3.sum())

237
237
237


In [265]:
## 회생기업 라벨링

df["공시제목"].fillna(0,inplace=True)

회생_ind = df[df["공시제목"]== '회생절차개시결정'].index

df.loc[회생_ind,"target"] = 1
df["target"].fillna(0, inplace=True)

### set1
df.loc[회생_ind,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) & (df.loc[i,'공시제목']=='회생절차개시결정'):
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

# ### set2
df.loc[회생_ind,"target_2"] = 1

df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)

## set_3

df.loc[회생_ind,"target_3"] = 1

df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)

In [266]:
print(df.target_1.sum())
print(df.target_2.sum())
print(df.target_3.sum())

312
275
275


In [267]:
def get_index_with_50_or_more_nan_columns(df):
    
    # 결측치 컬럼 개수 계산
    nan_column_counts = df.isnull().sum(axis=1)
    
    # 결측치 컬럼 개수가 50개 이상인 인덱스 번호 추출
    index_with_50_or_more_nan_columns = nan_column_counts[nan_column_counts >= 50].index.to_list()
    print('제거행 수:' , len(index_with_50_or_more_nan_columns))

    df.drop(index=index_with_50_or_more_nan_columns,inplace=True)
    return df


In [268]:
len(get_index_with_50_or_more_nan_columns(df))

제거행 수: 1156


12921

In [269]:
print('부실데이터 수:',df[df['target_1']==1].shape[0])
print('부실데이터 수:',df[df['target_2']==1].shape[0])
print('부실데이터 수:',df[df['target_3']==1].shape[0])

부실데이터 수: 308
부실데이터 수: 273
부실데이터 수: 273


In [270]:
pd.set_option('display.max_rows',None)
print(df.isna().sum())
pd.reset_option('display.max_rows',None)

회사명                             0
거래소코드                           0
회계년도                            0
자산                            300
자본                            300
차입금의존도                        485
유보율                           485
현금비율                          485
부채비율                          485
당좌비율                          485
순운전자본비율                       485
이자비용                          300
영업손익                          300
감가상각비                         300
EBITDA                        487
총자본정상영업이익률                    485
자기자본순이익률                      485
총자본순이익률                       485
매출액총이익률                       485
금융비용부담률                       485
경영자본순이익률                      485
자본금회전률                        485
유형자산회전율                       485
재고자산회전률                       485
자기자본회전률                       485
타인자본회전률                       485
총자본회전률                        485
이익잉여금                         300
영업활동으로 인한 현금흐름                300
매출액           

In [271]:
# 상장일이 회계년도 보다 작은 데이터 삭제
df = df[df["회계년도"] > df["상장일"]]
df.reset_index(drop = True, inplace = True)

print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 10931 개
전체 기업 수 1509


In [272]:
# 이용한 컬럼 삭제 

# df.drop(['자산','이자비용','매출액','부채','금융원가','시가총액',"EBITDA",'공시제목','상장일'],axis=1,inplace=True)

In [273]:
# # 이자부당률이 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

# df.loc[4173, "이자부담률"] = -5.995505908521291

# # 부채상환계수rk 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

# df.loc[4173, "부채상환계수"] = -760.5303077054485

# # 대주주 지분율 없는 컬럼 다트에서 값 찾아서 대체 

# df.loc[903, "대주주지분율"] = 50                    # 드림티엔터테인먼트
# df.loc[2913, "대주주지분율"] = 27.83                # 아라온테크
# df.loc[3668, "대주주지분율"] = 49.8                 # 에이루트
# df.loc[3702, "대주주지분율"] = 18.5                 # 에이제이에스
# df.loc[4399, "대주주지분율"] = 23.84                # 와이즈파워
# df.loc[5542, "대주주지분율"] = 100                  # 제이테크놀로지
# df.loc[8155, "대주주지분율"] = 12.03                # 대한그린파워
# df.loc[8326, "대주주지분율"] = 14.94                # 디브이에스코리아
# df.loc[8807, "대주주지분율"] = 40.78                # 세미텍


# #  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 

# df.loc[1879, "유형자산증가율"] = 0                    # 비츠로시스
# df.loc[1880, "유형자산증가율"] = 0                    # 비츠로시스

# # 외국인 지분율 결측치 0으로 대체
# df["외국인지분율"].fillna(0, inplace=True)


# # 직원인건비총액 결측치 0으로 대체
# df["직원인건비총액"].fillna(0, inplace=True)

# # 합계_기말인원(명) 결측치 0으로 대체
# df["합계_기말인원(명)"].fillna(0, inplace=True)



# #  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 
# df.loc[1879, "유형자산증가율"] = 0                    # # 유형자산을 모두 처분하여 감가상각비가 없어 EBITDA값이 결측치인 14년도 영진코퍼레이션 결측치에 영업이익값으로 대체


# # 이자부당률이 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

# df.loc[4173, "이자부담률"] = -5.995505908521291

# # 부채상환계수rk 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

# df.loc[4173, "부채상환계수"] = -760.5303077054485

# # 대주주 지분율 없는 컬럼 다트에서 값 찾아서 대체 

# df.loc[903, "대주주지분율"] = 50                    # 드림티엔터테인먼트
# df.loc[2913, "대주주지분율"] = 27.83                # 아라온테크
# df.loc[3668, "대주주지분율"] = 49.8                 # 에이루트
# df.loc[3702, "대주주지분율"] = 18.5                 # 에이제이에스
# df.loc[4399, "대주주지분율"] = 23.84                # 와이즈파워
# df.loc[5542, "대주주지분율"] = 100                  # 제이테크놀로지
# df.loc[8155, "대주주지분율"] = 12.03                # 대한그린파워
# df.loc[8326, "대주주지분율"] = 14.94                # 디브이에스코리아
# df.loc[8807, "대주주지분율"] = 40.78                # 세미텍


# #  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 

# df.loc[1879, "유형자산증가율"] = 0                    # 비츠로시스
# df.loc[1880, "유형자산증가율"] = 0                    # 비츠로시스

# 외국인 지분율 결측치 0으로 대체
df["외국인지분율"].fillna(0, inplace=True)


# 직원인건비총액 결측치 0으로 대체
# df["직원인건비총액"].fillna(0, inplace=True)

# 합계_기말인원(명) 결측치 0으로 대체
df["합계_기말인원(명)"].fillna(0, inplace=True)



# # #매출액증가율 없는 컬럼 삭제

de_ind_1 = df[df[["매출액증가율"]].isna().any(axis=1)].index
df.drop(index=de_ind_1, axis=0, inplace=True)
 # 메드팩토 셀리드 큐리언트


 # 증가율 컬럼 당해 회계년도에 상장하여 전년도 자료가 없어 결측치로 뜨는 데이터 삭제   
de_ind = df[df[["총자본증가율"]].isna().any(axis=1)].index
df.drop(index=de_ind, axis=0, inplace=True)

# # 직원수변동률 결측치 0으로 대체 
# de_ind_2 = df[df[["직원수변동률%"]].isna().any(1)].index
# df.drop(index=de_ind_2, axis=0, inplace=True)
# #  비덴트 썬테크놀로지스

In [274]:
pd.set_option('display.max_rows',None)
print(df.isna().sum())
pd.reset_option('display.max_rows',None)

회사명                          0
거래소코드                        0
회계년도                         0
자산                           0
자본                           0
차입금의존도                       0
유보율                          0
현금비율                         0
부채비율                         0
당좌비율                         0
순운전자본비율                      0
이자비용                         0
영업손익                         0
감가상각비                        0
EBITDA                       1
총자본정상영업이익률                   0
자기자본순이익률                     0
총자본순이익률                      0
매출액총이익률                      0
금융비용부담률                      0
경영자본순이익률                     0
자본금회전률                       0
유형자산회전율                      0
재고자산회전률                      0
자기자본회전률                      0
타인자본회전률                      0
총자본회전률                       0
이익잉여금                        0
영업활동으로 인한 현금흐름               0
매출액                          0
연구개발비(IFRS연결)(천원)            0
금융원가                         0
금융수익    

In [275]:
# 결측치 제거(1개)
df.dropna(axis=0, inplace=True)

In [276]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 10673 개
전체 기업 수 1455
결측치 수 :  0


In [277]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '자본', '차입금의존도', '유보율', '현금비율', '부채비율',
       '당좌비율', '순운전자본비율', '이자비용', '영업손익', '감가상각비', 'EBITDA', '총자본정상영업이익률',
       '자기자본순이익률', '총자본순이익률', '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률',
       '유형자산회전율', '재고자산회전률', '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금',
       '영업활동으로 인한 현금흐름', '매출액', '연구개발비(IFRS연결)(천원)', '금융원가', '금융수익',
       '법인세비용차감전순이익', '당기순이익', '부채', '합계_기말인원(명)', '합계_년간총급여(천원)', '외국인지분율',
       '대주주지분율', '단기차입금', '유동성장기부채', '상장일', '산업군', 'CAPEX',
       '현금및현금성자산(요약)(IFRS연결)(백만원)', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율',
       '정상영업이익증가율', '시가총액', '장단기금리차', 'cpi', '업력', '누적수익성비율', 'TMD', '시총/총자산',
       'EBIT/총자산', '당기전기영업손익', 'abs(영업현금흐름-당기순이익)/매출액', '공시제목', 'target',
       'target_1', 'target_2', 'target_3'],
      dtype='object')


## 결측치 처리 완료
---

In [278]:
# 무한대 값 확인

print("양의 무한대 :",df[df.isin([np.inf]).any(axis=1)]["target_3"].value_counts())
print("음의 무한대 :", df[df.isin([-np.inf]).any(axis=1)]["target_3"].value_counts())

양의 무한대 : target_3
0    7
Name: count, dtype: int64
음의 무한대 : Series([], Name: count, dtype: int64)


In [279]:
df[df.isin([np.inf]).any(axis=1)]

,회사명,거래소코드,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,...,TMD,시총/총자산,EBIT/총자산,당기전기영업손익,abs(영업현금흐름-당기순이익)/매출액,공시제목,target,target_1,target_2,target_3
2400,(주)셀리드,299660,2021,58256.0,38761.0,27.16,696.63,281.94,50.30,1127.06,...,0.040804,8.201757,-0.224149,-9149.0,5.352035,0,0.0,0,0,0
4543,(주)원바이오젠,307280,2021,38043.0,26561.0,21.26,632.51,70.38,43.23,164.94,...,0.155157,1.945232,-0.076466,931.0,0.098301,0,0.0,0,0,0
5941,(주)카이노스메드,284620,2021,41957.0,14600.0,42.73,9.95,90.55,187.37,143.72,...,0.097723,6.672163,-0.263508,-10302.5,4.229844,0,0.0,0,0,0
6518,(주)큐리언트,115180,2021,57622.0,54076.0,1.52,755.28,668.50,6.56,1944.57,...,0.013170,4.671287,-0.420256,-22634.0,0.141424,0,0.0,0,0,0
6957,(주)파멥신,208340,2019,161347.0,67783.0,33.84,1861.66,1761.65,138.03,5190.04,...,0.350292,1.655455,-0.068207,-8509.0,inf,0,0.0,0,0,0
6958,(주)파멥신,208340,2020,133553.0,45715.0,42.50,542.31,44.35,192.14,117.88,...,0.296671,2.216937,-0.190613,-17805.5,165.400000,0,0.0,0,0,0
7782,(주)휴맥스홀딩스,028080,2016,221495.0,192680.0,11.24,3206.31,9.95,14.95,14.89,...,0.327753,0.396924,-0.003124,5587.0,0.384232,0,0.0,0,0,0


In [280]:
# 무한대 값 나오는 거 인단 삭제

df.drop(index=df[df.isin([np.inf]).any(axis=1)].index, axis = 0, inplace = True)


In [281]:
# # 이자부담률 무한대 값 삭제
# # 전년도 이자부담률 데이터가 0값이서 데이터 제거(648개)

# a = df[df["이자부담률"].isin([np.inf])].index.to_list()
# b = df[df["이자부담률"].isin([-np.inf])].index.to_list()
# c = a + b
# df.drop(index=c, inplace=True)

In [282]:
# 유형자산증가율 무한대 값 삭제
# 전년도 유형자산 데이터가 0값이서 데이터 제거(2개)


a = df[df["유형자산증가율"].isin([np.inf])].index.to_list()
b = df[df["유형자산증가율"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [283]:
# # 현금흐름 대 매출액 무한대 값 삭제
# # 전년도 매출액 데이터가 0값이서 데이터 제거(2개)

# a = df[df["현금흐름 대 매출액"].isin([np.inf])].index.to_list()
# b = df[df["현금흐름 대 매출액"].isin([-np.inf])].index.to_list()
# c = a + b
# df.drop(index=c, inplace=True)

In [284]:
# 매출액증가율 무한대 값 삭제
# 전년도 매출액 데이터가 0값이서 데이터 제거

a = df[df["매출액증가율"].isin([np.inf])].index.to_list()
b = df[df["매출액증가율"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [285]:
# # ROA변화율 무한대 값 삭제
# # 전년도 ROA 데이터가 0값이서 데이터 제거

# a = df[df["ROA변화율"].isin([np.inf])].index.to_list()
# b = df[df["ROA변화율"].isin([-np.inf])].index.to_list()
# c = a + b
# df.drop(index=c, inplace=True)

In [286]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 10666 개
전체 기업 수 1451
결측치 수 :  0



## 무한대 값 처리 완료
---

In [287]:
# 산업군이 "금융 및 보험업"인 데이터 삭제

df[df["산업군"].str.contains("금융")][["회사명", "회계년도", "target_3", "산업군"]]

,회사명,회계년도,target_3,산업군
8013,글로벌텍스프리(주),2018,0,금융 및 보험업
8014,글로벌텍스프리(주),2019,0,금융 및 보험업
8015,글로벌텍스프리(주),2020,0,금융 및 보험업
8016,글로벌텍스프리(주),2021,0,금융 및 보험업
10815,해성산업(주),2012,0,금융 및 보험업
10816,해성산업(주),2013,0,금융 및 보험업
10817,해성산업(주),2014,0,금융 및 보험업
10818,해성산업(주),2015,0,금융 및 보험업
10819,해성산업(주),2016,0,금융 및 보험업
10820,해성산업(주),2017,0,금융 및 보험업


In [288]:
# del_col = df[df["산업군"].str.contains("금융")][["회사명", "회계년도", "target_3", "산업군"]].index

# df.drop(index=del_col, axis=0, inplace=True)
# df.reset_index(drop=True, inplace=True)


In [289]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 10666 개
전체 기업 수 1451
결측치 수 :  0


# 원본 데이터 결측치 처리 완료
---

In [290]:
# # 타겟 1인 기업들 0값 지워주기

# # 부실정의 컬럼 
# df['sum'] = df['target_1']+df['target_2']+df['target_3']

# ## 임시 부실데이터저장
# temp = pd.DataFrame()
# for i in df['회사명'].unique():
#     a = df[(df['회사명']==i) & df['sum']!=0]
#     temp = pd.concat([temp,a],axis=0)
# temp.drop(columns=['sum'],axis=1,inplace=True)
# print(temp.shape)



In [291]:
# for i in temp['회사명'].unique():
#     df = df[~(df['회사명']==i)]

# df = pd.concat([df,temp],axis=0)

In [292]:
# df.drop(columns=["sum"], axis=1, inplace=True)
# df.reset_index(drop=True,inplace=True)
# print(df.shape)


In [293]:
df["target_3"].value_counts()

target_3
0    10432
1      234
Name: count, dtype: int64

In [294]:
print(len(df[df["target_3"]==1]["회사명"].unique()))
print(len(df[df["target_3"]==0]["회사명"].unique()))

149
1414


In [295]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 10666 개
전체 기업 수 1451
결측치 수 :  0


In [296]:
df.describe()

,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,이자비용,...,누적수익성비율,TMD,시총/총자산,EBIT/총자산,당기전기영업손익,abs(영업현금흐름-당기순이익)/매출액,target,target_1,target_2,target_3
count,10666.000000,1.066600e+04,1.066600e+04,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000,...,10666.000000,10666.000000,10666.000000,10666.000000,1.066600e+04,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000
mean,2016.862366,2.217365e+05,1.122874e+05,21.195172,1189.803286,77.035832,111.878670,253.996550,20.916737,1979.288299,...,0.106414,0.919258,1.262760,0.000674,8.387764e+03,0.258392,0.013313,0.024939,0.021939,0.021939
std,2.873810,7.936189e+05,2.002822e+05,19.271079,1630.887238,258.793998,381.710359,593.255611,29.997388,4801.119834,...,0.759304,2.177647,1.692478,0.180740,2.903190e+04,2.486601,0.114618,0.155947,0.146491,0.146491
min,2012.000000,1.629000e+03,-5.429690e+05,0.000000,-1490.240000,0.000000,0.000000,1.810000,-1243.210000,-28.000000,...,-38.243959,0.001306,0.012305,-4.745856,-1.665870e+05,0.000004,0.000000,0.000000,0.000000,0.000000
25%,2014.000000,6.305100e+04,3.600275e+04,4.652500,314.962500,12.960000,30.190000,78.485000,3.032500,140.000000,...,-0.025992,0.177860,0.474508,-0.027712,-1.038750e+03,0.031233,0.000000,0.000000,0.000000,0.000000
50%,2017.000000,1.093940e+05,6.428250e+04,18.770000,773.015000,34.445000,65.470000,136.210000,21.690000,727.000000,...,0.225504,0.431009,0.811878,0.028694,2.907750e+03,0.073447,0.000000,0.000000,0.000000,0.000000
75%,2019.000000,2.024285e+05,1.168628e+05,33.877500,1499.027500,82.627500,125.952500,265.170000,39.787500,2025.000000,...,0.442504,0.968925,1.444983,0.072434,9.856250e+03,0.157700,0.000000,0.000000,0.000000,0.000000
max,2021.000000,4.700146e+07,4.867369e+06,749.400000,35257.990000,21837.410000,25729.810000,42354.180000,94.820000,149627.000000,...,3.210667,72.854765,37.017833,3.066083,1.185388e+06,175.940945,1.000000,1.000000,1.000000,1.000000


In [297]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '자본', '차입금의존도', '유보율', '현금비율', '부채비율',
       '당좌비율', '순운전자본비율', '이자비용', '영업손익', '감가상각비', 'EBITDA', '총자본정상영업이익률',
       '자기자본순이익률', '총자본순이익률', '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률',
       '유형자산회전율', '재고자산회전률', '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금',
       '영업활동으로 인한 현금흐름', '매출액', '연구개발비(IFRS연결)(천원)', '금융원가', '금융수익',
       '법인세비용차감전순이익', '당기순이익', '부채', '합계_기말인원(명)', '합계_년간총급여(천원)', '외국인지분율',
       '대주주지분율', '단기차입금', '유동성장기부채', '상장일', '산업군', 'CAPEX',
       '현금및현금성자산(요약)(IFRS연결)(백만원)', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율',
       '정상영업이익증가율', '시가총액', '장단기금리차', 'cpi', '업력', '누적수익성비율', 'TMD', '시총/총자산',
       'EBIT/총자산', '당기전기영업손익', 'abs(영업현금흐름-당기순이익)/매출액', '공시제목', 'target',
       'target_1', 'target_2', 'target_3'],
      dtype='object')

In [298]:
df.to_csv("./datasets/전처리준비/코스닥결측치처리완료_00.csv", encoding="utf-8-sig")